In [1]:
# import all needed libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import pymssql
from datetime import datetime

import copy
%load_ext autotime
from datetime import datetime as dt
import googletrans    
from googletrans import Translator
translator = Translator()

time: 145 ms (started: 2022-04-29 06:56:08 +00:00)


In [2]:
# distilbert-base-uncased-finetuned-sst-2-english
# Trained on BERT base model as a teacher
from transformers import pipeline
classifier1 = pipeline("sentiment-analysis")

def bert_distilbert(text):
    text = str(text)
    if text == 'nan ' or text == '':
        return math.nan
    return classifier1(text)
# This a bert-base-multilingual-uncased model finetuned for sentiment analysis on product reviews
# in six languages: English, Dutch, German, French, Spanish and Italian
# It predicts the sentiment of the review as a number of stars (between 1 and 5)
classifier2 = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')  

def bert_multilingual(text):
    text = str(text)
    if text == 'nan ' or text == '':
        return math.nan
    return classifier2(text)
# The model was fine-tuned and evaluated on 15 data sets from diverse text sources to enhance 
# generalization across different types of texts (reviews, tweets, etc.)
classifier3 = pipeline('sentiment-analysis', model='siebert/sentiment-roberta-large-english')   

def roberta_english(text):
    text = str(text)
    if text == 'nan ' or text == '':
        return math.nan
    return classifier3(text)
# This is a roBERTa-base model trained on ~58M tweets and finetuned for sentiment analysis 
# with the TweetEval benchmark.
# Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive
classifier4 = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment')     

def roberta_tweet(text):
    text = str(text)
    if text == 'nan ' or text == '':
        return math.nan
    return classifier4(text)

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj= SentimentIntensityAnalyzer()
from sentifish import Sentiment
from afinn import Afinn
afn = Afinn()

def textblob(text):
    text = str(text)
    res = TextBlob(text)
    return res.sentiment.polarity

def vader(text):
    text = str(text)
    temp = sid_obj.polarity_scores(sentence)
    return temp['compound']

def sentifish(text):
    text = str(text)
    obj=Sentiment(text)
    polarity = obj.analyze( )
    return polarity

def afinn(text):
    text = str(text)
    return afn.score(text)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


time: 18.7 s (started: 2022-04-29 06:56:08 +00:00)


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy
    
def dataframe_display(text):
    doc = nlp(text)
    df = pd.DataFrame(columns=['token','.dep_','.pos_','.tag_','children','children.pos_','children.text'])
    for token in doc:
        c = []
        p = []
        t = []
        for children in token.children:
            c.append(children)
            p.append(children.pos_)
            t.append(children.text)
        df2 = {'token':token,
              '.dep_':token.dep_,
              '.pos_':token.pos_,
              '.tag_':token.tag_,
              'children':c,
              'children.pos_':p,
              'children.text':t}
        df = df.append(df2, ignore_index = True)
    display(pd_centered(df))
    
def pos_display(text):
    doc = nlp(text)
    sentence_spans = list(doc.sents)
    options = {'distance': 100}
    displacy.render(sentence_spans, style='dep',jupyter=True,options=options)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


time: 819 ms (started: 2022-04-29 06:56:27 +00:00)


In [4]:
def print_list(lis):
    print("Length of list:",len(lis))
    for i in range(len(lis)):
        print(lis[i])
        
def pd_centered(df):
    return df.style.set_table_styles([
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "td", "props": [("text-align", "center")]}])

time: 622 µs (started: 2022-04-29 06:56:27 +00:00)


In [5]:
connection1 = pymssql.connect(server='100.22.1.57',user='aravindh',password='c4o@ER',database ='djuboreview',port='1443')

query1 ='select hotelcode,review_content,review_header,sentiment_score,polarity,properties,reviewid_new,id,review_score,language,Websitecode from djuboreview.dbo.reviewdetails_bak with (nolock) where id= 2404466'# and dtcollected ='+dt.today().strftime("'%Y-%m-%d'")

df=pd.read_sql(query1,connection1)



time: 740 ms (started: 2022-04-29 06:56:27 +00:00)


In [6]:
df.shape

(1, 11)

time: 3.73 ms (started: 2022-04-29 06:56:28 +00:00)


In [7]:
df['review_content1']=df['review_content']

df.loc[df['language']!='en','review_content1']= df[df['language']!='en']['review_content1'].apply(lambda x: translator.translate(x, dest='en').text)

time: 4.84 ms (started: 2022-04-29 06:56:28 +00:00)


In [8]:
df.loc[df['review_content1'].isnull(),'review_content1']='0'

df['Sentiment_Score']=0.0

df['Sentiment_Score']=df['Sentiment_Score'].astype(float)

vb=df[(df['review_content1'].str.contains("positive",case=False)) & (df['review_content1'].str.contains("negative",case=False))|(df['review_content1']=='0')|(df['review_content1']=='There are no comments available for this review')]

time: 26.3 ms (started: 2022-04-29 06:56:28 +00:00)


In [9]:
vb=vb.reset_index()

time: 1.92 ms (started: 2022-04-29 06:56:28 +00:00)


In [10]:
vb.drop('index',inplace=True,axis=1)

time: 2.24 ms (started: 2022-04-29 06:56:28 +00:00)


In [11]:
for i in range(0,len(vb)):
    if vb['Websitecode'][i]==2 or vb['Websitecode'][i]==5 or vb['Websitecode'][i]==12:
        a = list(np.arange(start=1, stop=10, step=0.8))
        OldMax = max(a)
        OldMin = min(a)

        NewMax = round(1)
        NewMin = -1

        OldRange = (OldMax - OldMin)  
        NewRange = (NewMax - NewMin) 

        vb['Sentiment_Score'][i] = (((vb['review_score'][i] - OldMin) * NewRange) / OldRange) + NewMin
    else:
        a = list(np.arange(start=1, stop=5, step=0.8))
        OldMax = max(a)
        OldMin = min(a)

        NewMax = 1
        NewMin = -1

        OldRange = (OldMax - OldMin)  
        NewRange = (NewMax - NewMin) 

        vb['Sentiment_Score'][i] = (((vb['review_score'][i] - OldMin) * NewRange) / OldRange) + NewMin



time: 1.35 ms (started: 2022-04-29 06:56:28 +00:00)


In [12]:
oo=df[~df['id'].isin(vb['id'])]

df=pd.concat([oo,vb])

df=df.reset_index()

time: 6.17 ms (started: 2022-04-29 06:56:28 +00:00)


In [13]:
cd = pd.DataFrame(columns = ['hotelcode', 'review_header','review_content',
         'reviewid_new', 'id','review_score','row','Reviewcontent','ReviewContentPos','ReviewContentNeg','ReviewContentNeu','Sentiment_Score'])

counter = 0
for i in range(df.shape[0]):
#     print(i)
    if df.loc[i,'review_content1']=='kkk':
        df.loc[i,'review_content1'] = math.nan
#         print(i)
    try:
        if math.isnan(df.loc[i,'review_content1']):
            counter += 1
            continue
    except:
        a = 1
    
    text = df.loc[i,'review_content1']
    now = text
    
    now = now.split('\n')
    pos = math.nan
    neg = math.nan
    neu = math.nan
            
    for j in range(len(now)):
        x = now[j].split()
        if len(x)==0:
            continue
        if x[0] == '[Positive]:':
            pos = ' '.join(x[1:])
        elif x[0] == '[Negative]:':
            neg = ' '.join(x[1:])
        else:
            neu = ' '.join(x)
    df2 = {'row':i,
           'Reviewcontent':df.loc[i,'review_content1'],
           'hotelcode':df.loc[i,'hotelcode'],
           'review_header':df.loc[i,'review_header'],
           'reviewid_new':df.loc[i,'reviewid_new'],
           'review_content':df.loc[i,'review_content'],
           'Sentiment_Score':df.loc[i,'Sentiment_Score'],
           'id':df.loc[i,'id'],
           'review_score':df.loc[i,'review_score'],
           'ReviewContentPos':pos,
           'ReviewContentNeg':neg,
           'ReviewContentNeu':neu}
    cd = cd.append(df2, ignore_index = True)
print(counter)

0
time: 11.6 ms (started: 2022-04-29 06:56:28 +00:00)


In [14]:
cd["ReviewContentPos"] = cd["ReviewContentPos"].astype(str)
cd["ReviewContentNeg"] = cd["ReviewContentNeg"].astype(str)
cd["ReviewContentNeu"] = cd["ReviewContentNeu"].astype(str)

time: 1.97 ms (started: 2022-04-29 06:56:28 +00:00)


In [15]:
import re
# Remove Punctuations without fullstop
rem_pun = '!"#$%&\'()*+-/:;<=>?@[\\]^_`{|}~'
def remove_punctuation_without_full(text):
    punctuationfree="".join([i if i not in rem_pun else ' ' for i in text])
    return punctuationfree

# Remove Numbers
def remove_numbers(text):
    pattern = r'[0-9]'
    line = re.sub(pattern,' ', text)
    return line

# Remove non ascii characters
def remove_non_ascii(text):
    now = text
    now = now.strip().split()
    now = [''.join([i for i in word if ord(i) < 128]) for word in now]
    text = ""
    for i in range(len(now)):
        text = text + now[i] + " "
    return text

# Convert multiple spaces to single space
def convert_multiple_spaces(text):
    answer = re.sub(' +', ' ', text)
    return answer

# Remove first and  end spaces
def remove_first_end_spaces(string):
    return "".join(string.rstrip().lstrip())

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
def lemma(text):
    now = text
    now = now.strip().split()
    now = [lemmatizer.lemmatize(word) for word in now]
    text = ""
    for i in range(len(now)):
        text = text + now[i] + " "
    return text

time: 1.6 ms (started: 2022-04-29 06:56:28 +00:00)


In [16]:
print('Preprocessing Started...')

cd['clean_Reviewcontent'] = cd['Reviewcontent'].apply(lambda x:remove_punctuation_without_full(x))
cd['clean_Reviewcontent'] = cd['clean_Reviewcontent'].apply(lambda x:remove_numbers(x))
cd['clean_Reviewcontent'] = cd['clean_Reviewcontent'].apply(lambda x:remove_non_ascii(x))
cd['clean_Reviewcontent'] = cd['clean_Reviewcontent'].apply(lambda x:lemma(x))
cd['clean_Reviewcontent'] = cd['clean_Reviewcontent'].apply(lambda x:convert_multiple_spaces(x))
cd['clean_Reviewcontent'] = cd['clean_Reviewcontent'].apply(lambda x:remove_first_end_spaces(x))

cd['clean_ReviewContentPos'] = cd['ReviewContentPos'].apply(lambda x:remove_punctuation_without_full(x))
cd['clean_ReviewContentPos'] = cd['clean_ReviewContentPos'].apply(lambda x:remove_numbers(x))
cd['clean_ReviewContentPos'] = cd['clean_ReviewContentPos'].apply(lambda x:remove_non_ascii(x))
cd['clean_ReviewContentPos'] = cd['clean_ReviewContentPos'].apply(lambda x:lemma(x))
cd['clean_ReviewContentPos'] = cd['clean_ReviewContentPos'].apply(lambda x:convert_multiple_spaces(x))
cd['clean_ReviewContentPos'] = cd['clean_ReviewContentPos'].apply(lambda x:remove_first_end_spaces(x))

cd['clean_ReviewContentNeg'] = cd['ReviewContentNeg'].apply(lambda x:remove_punctuation_without_full(x))
cd['clean_ReviewContentNeg'] = cd['clean_ReviewContentNeg'].apply(lambda x:remove_numbers(x))
cd['clean_ReviewContentNeg'] = cd['clean_ReviewContentNeg'].apply(lambda x:remove_non_ascii(x))
cd['clean_ReviewContentNeg'] = cd['clean_ReviewContentNeg'].apply(lambda x:lemma(x))
cd['clean_ReviewContentNeg'] = cd['clean_ReviewContentNeg'].apply(lambda x:convert_multiple_spaces(x))
cd['clean_ReviewContentNeg'] = cd['clean_ReviewContentNeg'].apply(lambda x:remove_first_end_spaces(x))

cd['clean_ReviewContentNeu'] = cd['ReviewContentNeu'].apply(lambda x:remove_punctuation_without_full(x))
cd['clean_ReviewContentNeu'] = cd['clean_ReviewContentNeu'].apply(lambda x:remove_numbers(x))
cd['clean_ReviewContentNeu'] = cd['clean_ReviewContentNeu'].apply(lambda x:remove_non_ascii(x))
cd['clean_ReviewContentNeu'] = cd['clean_ReviewContentNeu'].apply(lambda x:lemma(x))
cd['clean_ReviewContentNeu'] = cd['clean_ReviewContentNeu'].apply(lambda x:convert_multiple_spaces(x)) 
cd['clean_ReviewContentNeu'] = cd['clean_ReviewContentNeu'].apply(lambda x:remove_first_end_spaces(x)) 

print('Preprocessing Done!')

Preprocessing Started...
Preprocessing Done!
time: 1.59 s (started: 2022-04-29 06:56:28 +00:00)


In [17]:
raw0 = cd.loc[:,'clean_Reviewcontent'].to_list()
raw1 = cd.loc[:,'clean_ReviewContentPos'].to_list()
raw2 = cd.loc[:,'clean_ReviewContentNeg'].to_list()
raw3 = cd.loc[:,'clean_ReviewContentNeu'].to_list()

time: 1.07 ms (started: 2022-04-29 06:56:30 +00:00)


In [18]:
rem_all = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
def remove_punctuation_without_gap(text):
    punctuationfree="".join([i for i in text if i not in rem_all])
    # punctuationfree = punctuationfree.replace(' ','')
    return punctuationfree

from nltk.corpus import stopwords
stopwords_lst = stopwords.words('english')
sw = set(['dont','nothing','couldnt','wouldnt','doesnt','mustnt',
          'couldn','wouldn','doesn','mustn','neednt','needn','wont',
          'unfortunately','havent','arent','haven','aren','cant',
          'didnt','didn','shouldnt','shouldn','werent','weren',
          'shant','don','cannot','sorry','awfully','not'])
for word in stopwords_lst:
    sw.add(remove_punctuation_without_gap(word))
    
import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords = list(set(stopwords_list.decode().splitlines()))
for word in stopwords:
    sw.add(remove_punctuation_without_gap(word))

def remove_stopwords(text):
    now = text
    now = now.strip().split()
    now = [word for word in now if word not in sw and len(word)>2]
    text = ""
    for i in range(len(now)):
        text = text + now[i] + " "
    return text

time: 48 ms (started: 2022-04-29 06:56:30 +00:00)


In [19]:
def aspect_based1(sentences):
#     print(sentences)
    aspects = []
    for sentence in sentences:
        if sentence == '':
            continue
        doc = nlp(sentence)
        descriptive_term = []
        target = []
        adv = []
        negation = []
        descriptive_term_i = []
        target_i = []
        adv_i = []
        negation_i = []
        sep = []
        sep_i = []
        for token in doc:
            if token.pos_ == 'PUNCT' or token.pos_ == 'CCONJ':
                sep.append(token.text.lower())
                sep_i.append(token.i)
                continue
            if (token.dep_ == 'neg' and token.pos_ =='PART') or token.text.lower()=='no':
                negation.append(token.text.lower())
                negation_i.append(token.i)
                continue
            if remove_punctuation_without_gap(token.text.lower()) in sw:
                continue
            if token.pos_ == 'VERB' or token.pos_ == 'ADV':
                adv.append(token.text.lower())
                adv_i.append(token.i)
            if token.pos_ == 'NOUN':
                target.append(token.text.lower())
                target_i.append(token.i)
            if token.pos_ == 'ADJ':
                descriptive_term.append(token.text.lower())
                descriptive_term_i.append(token.i)
            if token.pos_ == 'ADJ':
                prepend = {}
                for child in token.children:
                    if remove_punctuation_without_gap(child.text.lower()) in sw:
                        continue
                    if child.pos_ == 'ADJ' or child.pos_ == 'ADV':
                        prepend[child.text.lower()] = child.i
                        if child.text.lower() in descriptive_term:
                            ind = descriptive_term.index(child.text.lower())
                            descriptive_term.pop(ind)
                            descriptive_term_i.pop(ind)
                        if token.text.lower() in descriptive_term:
                            ind = descriptive_term.index(token.text.lower())
                            descriptive_term.pop(ind)
                            descriptive_term_i.pop(ind)
                if len(prepend) > 0:
                    prepend[token.text.lower()] = token.i
                    prepend_sorted = sorted(prepend.items(), key=lambda x: x[1]) 
                    text_to_add = ''
                    temp = []
                    for i in range(len(prepend_sorted)-1):
                        text_to_add += prepend_sorted[i][0] + " "
                        temp.append(prepend_sorted[i][1])
                    text_to_add += prepend_sorted[len(prepend_sorted)-1][0]
                    temp.append(prepend_sorted[len(prepend_sorted)-1][1])
                    descriptive_term.append(text_to_add)
                    descriptive_term_i.append(temp)
        to_del = []
        for i in range(len(adv_i)):
            now = adv_i[i]
            for j in range(len(descriptive_term_i)):
                test = descriptive_term_i[j]
                if type(test) == list:
                    for k in range(len(test)):
                        if now == test[k]:
                            to_del.append(now)
                else:
                    if now == test:
                        to_del.append(now)
        for i in range(len(to_del)):
            ind = adv_i.index(to_del[i])
            adv.pop(ind)
            adv_i.pop(ind)
        
        for i in range(len(descriptive_term_i)):
            test = descriptive_term_i[i]
            if type(test) == list:
                descriptive_term_i[i] = np.mean(test)
                
        for i in range(len(negation_i)):
            test = negation_i[i]
            if type(test) == list:
                negation_i[i] = np.mean(test)
                
        for i in range(len(adv_i)):
            test = adv_i[i]
            if type(test) == list:
                adv_i[i] = np.mean(test)
                
        for i in range(len(negation_i)):
            min_dis = 1000
            now = -1
            typ = -1
            for j in range(len(descriptive_term_i)):
                if descriptive_term_i[j] > negation_i[i] and descriptive_term_i[j] - negation_i[i] < min_dis:
                    min_dis = descriptive_term_i[j] - negation_i[i]
                    now = j
                    typ = 1
            for j in range(len(adv_i)):
                if adv_i[j] > negation_i[i] and adv_i[j] - negation_i[i] < min_dis:
                    min_dis = adv_i[j] - negation_i[i]
                    now = j
                    typ = 2
            if now == -1:
                for j in range(len(descriptive_term_i)):
                    if abs(descriptive_term_i[j] - negation_i[i]) < min_dis:
                        min_dis = descriptive_term_i[j] - negation_i[i]
                        now = j
                        typ = 1
                for j in range(len(adv_i)):
                    if abs(adv_i[j] - negation_i[i]) < min_dis:
                        min_dis = adv_i[j] - negation_i[i]
                        now = j
                        typ = 2
            if typ == 1:
                descriptive_term[now] = negation[i] + ' ' + descriptive_term[now]
            if typ == 2:
                adv[now] = negation[i] + ' ' + adv[now]
                    
        aspects.append({'sentence':sentence,'aspect': target,
                        'description': descriptive_term,'adv':adv,
                        'negation':negation,'aspect_i': target_i,
                        'description_i': descriptive_term_i,'adv_i':adv_i,
                        'negation_i':negation_i,'sep':sep,'sep_i':sep_i})
#     print_list(aspects)
    return aspects

time: 3.82 ms (started: 2022-04-29 06:56:30 +00:00)


In [20]:
def combine_dictionary(a,b):
    final = {}
    if len(a) == 0:
        return b
    elif len(b) == 0:
        return a
    for itm in a:
        final[itm] = [a[itm]]
    for itm in b:
        if itm in final:
            now = final[itm]
            now.append(b[itm])
        else:
            final[itm] = [b[itm]]
    for itm in final:
        now = final[itm]
        score = np.mean(now)
        final[itm] = score
    return final

time: 2 ms (started: 2022-04-29 06:56:30 +00:00)


In [21]:
def aspect_relation1(dict_now):
    ans = {}
    return ans

def aspect_relation2(dict_now,column):
    ans = {}
    target = dict_now['aspect']
    sen = dict_now['sentence']
    if column == 'POSITIVE':
        for i in range(len(target)):
            ans[target[i]] = 1
    elif column == 'NEGATIVE':
        for i in range(len(target)):
            ans[target[i]] = -1
    else:
        s1 = bert_distilbert(sen)[0]
        r1 = 1000
        if s1['label'] == 'POSITIVE':
            r1 = s1['score']
        elif s1['label'] == 'NEGATIVE':
            r1 = -s1['score']
            
        s2 = roberta_english(sen)[0]
        r2 = 1000
        if s2['label'] == 'POSITIVE':
            r2 = s2['score']
        elif s2['label'] == 'NEGATIVE':
            r2 = -s2['score']
            
        score = round((r1+r2)/2,5)
        for i in range(len(target)):
            ans[target[i]] = score
            #print(ans)
    return ans

def aspect_relation3(dict_now):
    ans = {}
    target = dict_now['aspect']
    sen = dict_now['sentence']

    s1 = bert_distilbert(sen)[0]
    r1 = 1000
    if s1['label'] == 'POSITIVE':
        r1 = s1['score']
    elif s1['label'] == 'NEGATIVE':
        r1 = -s1['score']

    s2 = roberta_english(sen)[0]
    r2 = 1000
    if s2['label'] == 'POSITIVE':
        r2 = s2['score']
    elif s2['label'] == 'NEGATIVE':
        r2 = -s2['score']

    score = round((r1+r2)/2,5)
    ans[target[0]] = score
    #print(ans)
    return ans

def aspect_relation4(dict_now):
    ans = {}
    target = dict_now['aspect']
    sen = dict_now['sentence']

    s1 = bert_distilbert(sen)[0]
    r1 = 1000
    if s1['label'] == 'POSITIVE':
        r1 = s1['score']
    elif s1['label'] == 'NEGATIVE':
        r1 = -s1['score']

    s2 = roberta_english(sen)[0]
    r2 = 1000
    if s2['label'] == 'POSITIVE':
        r2 = s2['score']
    elif s2['label'] == 'NEGATIVE':
        r2 = -s2['score']

    score = round((r1+r2)/2,5)
    for i in range(len(target)):
        ans[target[i]] = score
        #print(ans)
    return ans

def aspect_relation5(dict_now):
    ans = {}
    target = dict_now['aspect']
    sen = dict_now['sentence']

    s1 = bert_distilbert(sen)[0]
    r1 = 1000
    if s1['label'] == 'POSITIVE':
        r1 = s1['score']
    elif s1['label'] == 'NEGATIVE':
        r1 = -s1['score']

    s2 = roberta_english(sen)[0]
    r2 = 1000
    if s2['label'] == 'POSITIVE':
        r2 = s2['score']
    elif s2['label'] == 'NEGATIVE':
        r2 = -s2['score']

    score = round((r1+r2)/2,5)
    ans[target[0]] = score
    #print(ans)
    return ans

def aspect_relation6(dict_now):
    ans = {}
    target = dict_now['aspect']
    target_i = dict_now['aspect_i']
    words1 = dict_now['description']
    words2 = dict_now['adv']
    wordsi_1 = dict_now['description_i']
    wordsi_2 = dict_now['adv_i']
    wordsf_1 = []
    wordsf_2 = []
    for i in range(len(wordsi_1)):
        now = wordsi_1[i]
        if type(now) == list:
            wordsf_1.append(np.mean(now))
        else:
            wordsf_1.append(now)
            
    for i in range(len(wordsi_2)):
        now = wordsi_2[i]
        if type(now) == list:
            wordsf_2.append(np.mean(now))
        else:
            wordsf_2.append(now)
    X = target
    XX = target_i
    Y = words1 + words2
    YY = wordsf_1 + wordsf_2
    
    y = copy.deepcopy(Y)
    yy = copy.deepcopy(YY)
    ans_list = []
    ind_list = []
    for i in range(len(XX)):
        min_dis = 1000
        now = 0
        for j in range(len(YY)):
            if abs(YY[j]-XX[i]) < min_dis:
                min_dis = abs(YY[j]-XX[i])
                now = j
        ans_list.append(Y[now])
        ind_list.append(now)
    ind_list = np.sort(list(set(ind_list)))[::-1].tolist()
    
    for i in range(len(ind_list)):
        index = ind_list[i]
        y.pop(index)
        yy.pop(index)
    
    relation = []
    for i in range(len(ans_list)):
        relation.append([target[i],ans_list[i]])
    
    for i in range(len(yy)):
        min_dis = 1000
        now = 0
        for j in range(len(XX)):
            if abs(yy[i]-XX[j]) < min_dis:
                min_dis = abs(yy[i]-XX[j])
                now = j
        relation.append([X[now],y[i]])
        
    
    for i in range(len(target)):
        ans[target[i]] = []
    for i in range(len(relation)):
        word = relation[i][0]
        sen = relation[i][1]
        
        s1 = bert_distilbert(sen)[0]
        r1 = 1000
        if s1['label'] == 'POSITIVE':
            r1 = s1['score']
        elif s1['label'] == 'NEGATIVE':
            r1 = -s1['score']

        s2 = roberta_english(sen)[0]
        r2 = 1000
        if s2['label'] == 'POSITIVE':
            r2 = s2['score']
        elif s2['label'] == 'NEGATIVE':
            r2 = -s2['score']

        score = round((r1+r2)/2,5)
        
        list_now = ans[word]
        list_now.append(score)
        ans[word] = list_now
    
    for itm in ans:
        value = ans[itm]
        value_mean = np.mean(value)
        ans[itm] = value_mean
       
    
    return ans

time: 3.92 ms (started: 2022-04-29 06:56:30 +00:00)


In [22]:
def type_detection(ab,column):
    rows = len(ab)
    final = {}
    for i in range(rows):
        dict_now = ab[i]
        NC = len(dict_now['aspect'])
        DC = len(dict_now['description']) + len(dict_now['adv'])
        if NC == 0:
#             print('Type 1')
            ans = aspect_relation1(dict_now)
            final = combine_dictionary(ans,final)
        elif DC == 0:
#             print('Type 2')
            ans = aspect_relation2(dict_now,column)
#             Print(column)
            final = combine_dictionary(ans,final)
        elif NC == 1 and DC == 1:
#             print('Type 3')
            ans = aspect_relation3(dict_now)
            final = combine_dictionary(ans,final)
        elif NC > 1 and DC == 1:
#             print('Type 4')
            ans = aspect_relation4(dict_now)
            final = combine_dictionary(ans,final)
        elif NC == 1 and DC > 1:
#             print('Type 5')
            ans = aspect_relation5(dict_now)
            final = combine_dictionary(ans,final)
        elif NC > 1 and DC > 1:
#             print('Type 6')
            ans = aspect_relation6(dict_now)
            final = combine_dictionary(ans,final)
        else:
            print('Type X')
    return final

time: 1.17 ms (started: 2022-04-29 06:56:30 +00:00)


In [23]:
helper = pd.read_csv('aspect_dictionary.csv')
#helper=helper[(helper['Class']!='Z')&(helper['Class']!='z')]
print(helper.shape)
helper['Phrase'] = helper['Phrase'].astype(str)
helper['Class'] = helper['Class'].astype(str)
keywords = list(set(helper.loc[:,'Class'].to_list()))
classes_dict = {}
for i in range(len(helper)):
    add = lemma(remove_first_end_spaces(helper.loc[i,'Phrase']))
    #print(add)
    add = remove_punctuation_without_full(add)
    add = remove_first_end_spaces(add)
    classes_dict[add] = helper.loc[i,'Class']

(1345, 2)
time: 39.2 ms (started: 2022-04-29 06:56:30 +00:00)


In [24]:
import gensim.downloader as api
from scipy import spatial
glove_model = api.load('glove-twitter-25')

def find_aspect(aspect,keywords):
    sim = []
    sample_glove_embedding1=glove_model[aspect.lower()]
    for i in range(len(keywords)):
        sample_glove_embedding2=glove_model[keywords[i].lower()]
        res = 1 - spatial.distance.cosine(sample_glove_embedding1, sample_glove_embedding2)
        sim.append(res)
    x = np.argmax(sim)
    return keywords[x]

time: 27.7 s (started: 2022-04-29 06:56:30 +00:00)


In [25]:
def find_class(aspect, keywords, classes_dict):
    aspect = lemma(remove_first_end_spaces(aspect))
    aspect = remove_punctuation_without_full(aspect)
    aspect = remove_first_end_spaces(aspect)
    if aspect in classes_dict:
        if classes_dict[aspect] == 'dump':
            ret = -1
            return ret
        return classes_dict[aspect]
    try:
        ret = find_aspect(aspect, keywords)
    except:
        ret = -1
    return ret

time: 611 µs (started: 2022-04-29 06:56:58 +00:00)


In [26]:
def collate_classes(dict_now,keywords,classes_dict):
    ans = {}
    for itm in dict_now:
        class_add = find_class(itm,keywords,classes_dict)
        if class_add == -1:
            continue
        if class_add in ans:
            now = ans[class_add]
            now.append(dict_now[itm])
            ans[class_add] = now
        else:
            ans[class_add] = [dict_now[itm]]
    for itm in ans:
        now = ans[itm]
        score = np.mean(now)
        ans[itm] = round(score,5)
        #print(now)
    return ans

time: 46.4 ms (started: 2022-04-29 06:56:58 +00:00)


In [27]:
def ensemble_sentiment(text):
    snip = text
    now = bert_distilbert(snip)[0]
    x = 0
    y = 0
    if now['label'] == 'POSITIVE':
        x += now['score']
    else:
        x -= now['score']
    temp = roberta_english(snip)[0]
    if now['label'] == 'POSITIVE':
        y += now['score']
    else:
        y -= now['score']
#     print("Overall Sentiment Score:",round((x+y)/2,5))
    return round((x+y)/2,5)

time: 13 ms (started: 2022-04-29 06:56:58 +00:00)


In [28]:
def find_aspect_class_relation(relation, td, keywords, classes_dict):
    for itm in td:
        relation[itm] = find_class(itm, keywords, classes_dict)
    return relation

time: 1.11 ms (started: 2022-04-29 06:56:58 +00:00)


In [29]:
import sys, os

# for i in range(cd.shape[0]):
for i in range(cd.shape[0]):
#     print('Run:',i)
    try:
        text = cd.loc[i,'clean_Reviewcontent']
        text_lst=[]
        if len(text)>1800:
            res_first, res_second = text[:len(text)//2],text[len(text)//2:]
            text_lst.append(res_first)
            text_lst.append(res_second)
        else:
            text_lst.append(text)
        score_lst=[]
        for text1 in text_lst:
            a_score = ensemble_sentiment(text1)
            score_lst.append(a_score)
        cd.loc[i,'Sentiment Score'] =sum(score_lst)/len(score_lst)
#         cd.loc[i,'Sentiment Score'] = ensemble_sentiment(text)
        pol_prop = {}
        index = i
        snip = raw1[index]
        text = snip
        text = snip.split('.')
        ab = aspect_based1(text)
        # pos_display(snip)
        # dataframe_display(snip)
        # ensemble_sentiment(snip)
        # polarity_ab(ab)
        
        relation = {}

        td = type_detection(ab,'POSITIVE')
        relation = find_aspect_class_relation(relation, td, keywords, classes_dict)
        c1 = collate_classes(td,keywords,classes_dict)
        #print(keywords)

        index = i
        snip = raw2[index]
        text = snip
        text = snip.split('.')
        ab = aspect_based1(text)
        # pos_display(snip)
        # dataframe_display(snip)
        # ensemble_sentiment(snip)
        # polarity_ab(ab)

        td = type_detection(ab,'NEGATIVE')
        relation = find_aspect_class_relation(relation, td, keywords, classes_dict)
        c2 = collate_classes(td,keywords,classes_dict)
        #print(td)

        index = i
        snip = raw3[index]
        text = snip
        text = snip.split('.')
        ab = aspect_based1(text)
        # pos_display(snip)
        # dataframe_display(snip)
        # ensemble_sentiment(snip)
        # polarity_ab(ab)

        td = type_detection(ab,'NEUTRAL')
        relation = find_aspect_class_relation(relation, td, keywords, classes_dict)
        c3 = collate_classes(td,keywords,classes_dict)
        pol_prop = combine_dictionary(pol_prop,c1)
        pol_prop = combine_dictionary(pol_prop,c2)
        pol_prop = combine_dictionary(pol_prop,c3)

#         for itm in relation:
#             if itm in classes_dict:
#                 continue
#             print(itm,'--->',relation[itm])
   
        cd.loc[i,'Relation'] = str(relation)
        cd.loc[i,'Properties'] = str(pol_prop)
        if len(pol_prop)>0:
            for j in range (0,len(pol_prop)):
                dictt = pol_prop
                if list(pol_prop.values())[j] < 0.0:
                    dictt.update({list(pol_prop.keys())[j]:[list(pol_prop.values())[j],'Negative']})
                elif list(pol_prop.values())[j] > 0.0:
                    dictt.update({list(pol_prop.keys())[j]:[list(pol_prop.values())[j],'Positive']})
                else:
                    dictt.update({list(pol_prop.keys())[j]:[list(pol_prop.values())[j],'Neutral']})


            cd.loc[i,'Strength'] = str(dictt)
        else:
            cd.loc[i,'Strength'] = {}
        print(str(dictt))
        exit()
        
#         print('---------------------------------')

    except Exception as e:
        print(i,e)
        continue
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

0 Must have equal len keys and value when setting with an iterable
time: 200 ms (started: 2022-04-29 06:56:58 +00:00)


In [30]:
cd.loc[cd['Sentiment_Score']==0.0,'Sentiment_Score']=cd['Sentiment Score']

cd=cd[['hotelcode', 'review_header', 'review_content', 'reviewid_new', 'id',
       'review_score', 'row', 'Reviewcontent', 'ReviewContentPos',
       'ReviewContentNeg', 'ReviewContentNeu', 'Sentiment_Score',
       'clean_Reviewcontent', 'clean_ReviewContentPos',
       'clean_ReviewContentNeg', 'clean_ReviewContentNeu',
       'Relation', 'Properties', 'Strength']]

cd['Sentiment_Score']=cd['Sentiment_Score'].round(2)

cd['Sentiment_Score']=cd['Sentiment_Score'].astype(float)

time: 5.6 ms (started: 2022-04-29 06:56:58 +00:00)


In [31]:
cd.columns

Index(['hotelcode', 'review_header', 'review_content', 'reviewid_new', 'id',
       'review_score', 'row', 'Reviewcontent', 'ReviewContentPos',
       'ReviewContentNeg', 'ReviewContentNeu', 'Sentiment_Score',
       'clean_Reviewcontent', 'clean_ReviewContentPos',
       'clean_ReviewContentNeg', 'clean_ReviewContentNeu', 'Relation',
       'Properties', 'Strength'],
      dtype='object')

time: 2.49 ms (started: 2022-04-29 06:56:58 +00:00)


In [32]:
cd.loc[cd['Sentiment_Score'].isnull(),'Sentiment_Score']=0.0
cd.loc[cd['Sentiment_Score']< 0,'polarity']="negative"
cd.loc[(cd['Sentiment_Score']> 0.10),'polarity']="positive"
cd.loc[(cd['Sentiment_Score']<=0.10) & (cd['Sentiment_Score']>= 0),'polarity']="neutral"

time: 4.43 ms (started: 2022-04-29 06:56:58 +00:00)


In [33]:
# cd['Properties'][2]

time: 1.54 ms (started: 2022-04-29 06:56:58 +00:00)


In [34]:
# cd[(cd['Properties']=='{}')]

time: 1.04 ms (started: 2022-04-29 06:56:58 +00:00)


In [35]:
cd.to_excel('Todays_sample_with_Aayush_code6.xlsx',index=False)

time: 58.5 ms (started: 2022-04-29 06:56:58 +00:00)


In [36]:
# cd

time: 240 µs (started: 2022-04-29 06:56:58 +00:00)


In [37]:
# cd['Properties'].unique()

time: 1.31 ms (started: 2022-04-29 06:56:58 +00:00)


In [38]:
# {'fnb & bar': 0.99856}

time: 1.27 ms (started: 2022-04-29 06:56:58 +00:00)


In [39]:
# pol_prop={'fnb & bar': 0.99856}

time: 978 µs (started: 2022-04-29 06:56:58 +00:00)


In [40]:
# list(pol_prop.values())[0]<0

time: 912 µs (started: 2022-04-29 06:56:58 +00:00)


In [41]:
# dictt

time: 880 µs (started: 2022-04-29 06:56:58 +00:00)


In [42]:
# dictt = pol_prop

# if [list(pol_prop.values())[0]<0]:
#     str_prop=dictt.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Negative']})
#     print(str_prop)
# elif [list(pol_prop.values())[0]>0]:
#     str_prop=dictt.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Positive']})
#     print(str_prop)
# else:
#     str_prop=dictt.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Neutral']})


time: 610 µs (started: 2022-04-29 06:56:58 +00:00)


In [43]:
# list(pol_prop.values())[j]

time: 617 µs (started: 2022-04-29 06:56:58 +00:00)


In [44]:
# [list(pol_prop.values())[0]]

time: 1.26 ms (started: 2022-04-29 06:56:58 +00:00)


In [45]:

# dict = pol_prop

# if [list(pol_prop.values())[0]<1]:
#     str_prop=dict.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Negative']})
# elif [list(pol_prop.values())[0]>1]:
#     str_prop=dict.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Positive']})
# else:
#     str_prop=dict.update({list(pol_prop.keys())[0]:[list(pol_prop.values())[0],'Neutral']})


time: 1.08 ms (started: 2022-04-29 06:56:58 +00:00)


In [46]:
# pol_prop

time: 743 µs (started: 2022-04-29 06:56:58 +00:00)


In [47]:
# list(pol_prop.values())[2]

time: 963 µs (started: 2022-04-29 06:56:58 +00:00)


In [48]:
# cd['Properties']

time: 504 µs (started: 2022-04-29 06:56:58 +00:00)


In [49]:
# helper[helper['Class']=='z']

time: 373 µs (started: 2022-04-29 06:56:58 +00:00)


In [50]:
# list(pol_prop.values())[0]

time: 439 µs (started: 2022-04-29 06:56:58 +00:00)


In [51]:
# cd['Properties']

time: 1.43 ms (started: 2022-04-29 06:56:58 +00:00)


In [52]:
# dict

# dict

# cd['Properties']

# cd[cd['Reviewcontent'].str.contains('There are no comments',case=False)]

# cd[cd['Reviewcontent'].str.contains('There are no comments',case=False)]

# cd[cd['Reviewcontent']=='']

# cd

# cd['Properties'][0]

# cd.loc[(cd['Reviewcontent'].str.contains("positive",case=False)) & (cd['Reviewcontent'].str.contains("negative",case=False)),'Sentiment Score']=cd['Sentiment_Score']

# aa=cd[['hotelcode', 'review_header', 'review_content', 'reviewid_new', 'id',
#        'review_score','Sentiment Score','Properties']]

# aa.to_excel('sample_aayush_code.xlsx',index=False)



# a = list(np.arange(start=-1, stop=1, step=0.05))
# OldMax = max(a)
# OldMin = min(a)

# NewMax = 5
# NewMin = 0

# OldRange = (OldMax - OldMin)  
# NewRange = (NewMax - NewMin)  

# ab['Sentiment Score'] = (((ab['Sentiment Score'] - OldMin) * NewRange) / OldRange) + NewMin
   



# ab





# cd.to_excel('sample_results_testing.xlsx',index=False)

# cd[cd['Sentiment Score']==0.0]

# cd[cd['Reviewcontent'].str.contains('[Positive]')]

# cd[cd['ReviewContentPos']=='it’s clean']

# cd[cd['ReviewContentPos'].notnull()]



time: 533 µs (started: 2022-04-29 06:56:58 +00:00)
